# Sprint8
## アンサンブル学習

**主要ライブラリインポート**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as itr
import random

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error


**データセット用意**

In [2]:
data_set=pd.read_csv("/Users/wakaichiaki/diveintocode-ml/diveintocode-ml/Week03/house-prices-advanced-regression-techniques/train.csv")

In [3]:
#説明変数として、GrLivAreaとYearBuilt,目的変数としてSalePrice
X,y=data_set[["GrLivArea","YearBuilt"]],data_set[["SalePrice"]]

#data_split(8:2)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

#標準化データ
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## 【問題1】ブレンディングのスクラッチ実装

**１つ目(重みをづけ)**

In [4]:
#モデル定義
Lr=LinearRegression()
Sr=SVR()
Dt=DecisionTreeRegressor()

In [5]:
#fit
Lr.fit(X_train,y_train)
Sr.fit(X_train,y_train)
Dt.fit(X_train,y_train)
#pred
y_Lr=Lr.predict(X_test)
y_Sr=Sr.predict(X_test).reshape(-1,1)
y_Dt=Dt.predict(X_test).reshape(-1,1)
blend_1 = (y_Lr + y_Sr + y_Dt) / 3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [6]:
#MSE
Lr_MSE_1=mean_squared_error(y_test,y_Lr)
Sr_MSE_1=mean_squared_error(y_test,y_Sr)
Dt_MSE_1=mean_squared_error(y_test,y_Dt)
blend_MSE_1=mean_squared_error(y_test,blend_1)
print(
    "MSE:\n",
    "LinearRegression:",Lr_MSE_1,"\n",
    "SVR:",Sr_MSE_1,"\n",
    "DecisionTreeRegressor",Dt_MSE_1,"\n",
    "blend:",blend_MSE_1
     )

MSE:
 LinearRegression: 2112604877.4575124 
 SVR: 7337001562.819469 
 DecisionTreeRegressor 2935567781.986682 
 blend: 2654042522.5426316


各モデルによる教師データと予測データのMSEから、SVRの予測精度が低いことがわかる。
２つ目では、SVRの影響を弱めるため重みを小さくしてみる。

In [7]:
#MSE割合
all_MSE=Lr_MSE_1+Sr_MSE_1+Dt_MSE_1
weight=[Lr_MSE_1/all_MSE, Sr_MSE_1/all_MSE, Dt_MSE_1/all_MSE]
weight

[0.17057530556654432, 0.5924019663470241, 0.23702272808643154]

In [8]:
#重み定義
w_1=0.001 #SVR
w_2_3=0.4
w_2,w_3=(1-w_1)/2-w_2_3,(1-w_1)/2+w_2_3 #LogisticRegression,DecisionTreeRegressor
#Blending
blend_1w=(y_Lr*w_2+y_Sr*w_1+y_Dt*w_3)/3
print("重み\nLr:{}\nSr:{}\nDt:{}".format(w_1,w_2,w_3))

重み
Lr:0.001
Sr:0.09949999999999998
Dt:0.8995


In [9]:
#MSE
blend_MSE_1w=mean_squared_error(y_test,blend_1w)
print("MSE\n","blend:",blend_MSE_1w)

MSE
 blend: 19189539718.078793


**２つ目(パラメータを調整してみる)**

In [10]:
#モデル定義
Lr=LinearRegression(normalize = True)
Sr=SVR(gamma="scale", kernel="linear")
Dt=DecisionTreeRegressor(max_depth=10)

In [11]:
#fit
Lr.fit(X_train,y_train)
Sr.fit(X_train,y_train)
Dt.fit(X_train,y_train)
#pred
y_Sr_2=Lr.predict(X_test)
y_Lr_2=Sr.predict(X_test).reshape(-1,1)
y_Dt_2=Dt.predict(X_test).reshape(-1,1)
#Blending
blend_2=(y_Lr_2*w_1+y_Sr_2*w_2+y_Dt_2*w_3)/3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
#MSE
Lr_MSE_2=mean_squared_error(y_test,y_Lr_2)
Sr_MSE_2=mean_squared_error(y_test,y_Sr_2)
Dr_MSE_2=mean_squared_error(y_test,y_Dt_2)
blend_MSE_2=mean_squared_error(y_test,blend_2)
print(
    "MSE\n",
    "LinearRegression:",Lr_MSE_2,"\n",
    "SVR:",Sr_MSE_2,"\n",
    "DecisionTreeRegressor",Dr_MSE_2,"\n",
    "blend:",blend_MSE_2
     )

MSE
 LinearRegression: 2336843021.583233 
 SVR: 2112604877.4575112 
 DecisionTreeRegressor 2754178981.067846 
 blend: 19284462704.87909


**3つ目(データセットを標準化してみる)**

In [13]:
#標準化したデータでのモデル定義
Lr_sc=LinearRegression()
Sr_sc=SVR(gamma="scale", kernel="linear")
Dr_sc=DecisionTreeRegressor(max_depth=10)

In [14]:
#fit
Lr.fit(X_train_sc,y_train)
Sr.fit(X_train_sc,y_train)
Dt.fit(X_train_sc,y_train)
#pred
y_Lr_sc=Lr.predict(X_test)
y_Sr_sc=Sr.predict(X_test).reshape(-1,1)
y_Dt_sc=Dt.predict(X_test).reshape(-1,1)
#Blending
blend_3=(y_Lr_sc*w_1+y_Sr_sc*w_2+y_Dt_sc*w_3)/3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#MSE
Lr_MSE_3=mean_squared_error(y_test,y_Lr_sc)
Sr_MSE_3=mean_squared_error(y_test,y_Sr_sc)
Dt_MSE_3=mean_squared_error(y_test,y_Dt_sc)
blend_MSE_3=mean_squared_error(y_test,blend_3)
print(
    "MSE\n",
    "SGDRegressor:",Lr_MSE_3,"\n",
    "LogisticRegression:",Sr_MSE_3,"\n",
    "DecisionTreeRegressor",Dt_MSE_3,"\n",
    "blend:",blend_MSE_3
     )

MSE
 SGDRegressor: 1.906301356908743e+16 
 LogisticRegression: 5171338044215.514 
 DecisionTreeRegressor 7446242290.767123 
 blend: 4864358083.619314


## 【問題2】バギングのスクラッチ実装

In [16]:
X_train_b=X_train.values
y_train=y_train.values
X_train_b_list=[]
y_train_b_list=[]
for i in range(4):
    idx=np.random.randint(0,len(X_train_b),size=292)
    X_train_b_list.append(X_train_b[idx,:])
    y_train_b_list.append(y_train[idx])

In [17]:
#予測値配列
y_pred_b=np.zeros((X_test.shape[0],4))
#学習から予測
for i in range(4):
    model=DecisionTreeRegressor(max_depth=3)
    model.fit(X_train_b_list[i],y_train_b_list[i])
    y_pred_b[:,i]=model.predict(X_test)

In [18]:
#baging
baging = y_pred_b.sum(axis=1) / 4

In [19]:
for i in range(4):
    print("モデル{}：{}".format(i+1,mean_squared_error(y_test_b, y_pred_b[:,i])))
print("baging：{}".format(mean_squared_error(y_test_b, baging)))

NameError: name 'y_test_b' is not defined

## 【問題3】スタッキングのスクラッチ実装

**ステージ0**

In [20]:
#データの用意
#データ分割(K=3)
X_train_s1,y_train_s1=X_train_b[:389,:],y_train[:389,:]
X_train_s2,y_train_s2=X_train_b[389:778,:],y_train[389:778,:]
X_train_s3,y_train_s3=X_train_b[778:,:],y_train[778:,:]

#クロスバリデーション
train_X_1,train_y_1=np.vstack([X_train_s2,X_train_s3]),np.vstack([y_train_s2,y_train_s3])
train_X_2,train_y_2=np.vstack([X_train_s1,X_train_s3]),np.vstack([y_train_s1,y_train_s3])
train_X_3,train_y_3=np.vstack([X_train_s1,X_train_s2]),np.vstack([y_train_s1,y_train_s2])

**学習フェーズ**

In [28]:
#モデルのインスタンス作成
model_1=LinearRegression(normalize=True)
model_2=SVR(gamma="scale", kernel="linear")
model_3=DecisionTreeRegressor(max_depth=5)

#モデル配列
model_list=[model_1,model_2,model_3]
train_Xl=[train_X_1,train_X_2,train_X_3]
train_yl=[train_y_1,train_y_2,train_y_3]

pred_ley1=np.array([])
for model in model_list:
    temp=np,array([])
    for tr_X, tr_y in zip(train_Xl,train_yl):
        model.fit(tr_X,tr_y)
        pred=model.predict(X_test)
        temp=np.append(temp,pred)
    pred_ley1=np.append(pred_ley1,temp)    

NameError: name 'array' is not defined

In [27]:
y_pred_ley1[0]

IndexError: index 0 is out of bounds for axis 0 with size 0